In [ ]:
import category_encoders as ce
import pandas as pd
import numpy as np
import os

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_csv('../data/03_primary/df_train.csv')

In [ ]:
df

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
encodings = {
    'bool': ['c_is_mobile', 'c_is_first_visit'],
    'ohe': ['c_device_category', 'c_operating_system', 'c_browser', 'c_traffic_source', 'c_traffic_medium'],
    'int': ['c_country', 'c_city', 'c_traffic_campaign', 'c_weekday', 'c_visit_start_hour']
}

In [ ]:
binary_encoder = ce.BinaryEncoder(cols=encodings['bool'])

In [ ]:
binary_encoder.fit(df)
binary_encoder.transform(df)

In [ ]:
df_cp = df.copy()
df_cp.iloc[39, 12] = None
df_cp.iloc[:, 12]

In [ ]:
binary_encoder.fit(df)
binary_encoder.transform(df_cp)

In [ ]:
binary_encoder.fit(df_cp)
binary_encoder.transform(df_cp)

In [ ]:
onehot_encoder = ce.OneHotEncoder(cols=encodings['ohe'], use_cat_names=True)

In [ ]:
onehot_encoder.fit(df)
onehot_encoder.transform(df)

In [ ]:
df_cp = df.copy()
df_cp.iloc[39, 3] = None
df_cp.iloc[:, 3]

In [ ]:
onehot_encoder.fit(df)
onehot_encoder.transform(df_cp)

In [ ]:
onehot_encoder.fit(df_cp)
onehot_encoder.transform(df_cp)

In [ ]:
ordinal_encoder = ce.OrdinalEncoder(cols=encodings['int'])

In [ ]:
ordinal_encoder.fit(df)
ordinal_encoder.transform(df)

In [ ]:
df_cp = df.copy()
df_cp.iloc[39, 7] = None
df_cp.iloc[:, 7]

In [ ]:
ordinal_encoder.fit(df)
ordinal_encoder.transform(df_cp)

In [ ]:
ordinal_encoder.fit(df_cp)
ordinal_encoder.transform(df_cp)

In [ ]:
df_cp2 = df.copy()

In [ ]:
binary_encoder = ce.BinaryEncoder(cols=encodings['bool'])
onehot_encoder = ce.OneHotEncoder(cols=encodings['ohe'], use_cat_names=True)
ordinal_encoder = ce.OrdinalEncoder(cols=encodings['int'])

In [ ]:
binary_encoder.fit(df_cp2)
df_cp2 = binary_encoder.transform(df_cp2)
onehot_encoder.fit(df_cp2)
df_cp2 = onehot_encoder.transform(df_cp2)
ordinal_encoder.fit(df_cp2)
df_cp2 = ordinal_encoder.transform(df_cp2)
df_cp2

---

In [ ]:
df.dtypes

In [ ]:
pd.to_datetime(df['i_visit_start_time'], unit='us')

In [ ]:
pd.to_datetime(df['i_visit_start_time'], unit='us').dt.weekday

In [ ]:
pd.to_datetime(df['i_visit_start_time'], unit='us').dt.hour

In [ ]:
df_cp2['c_weekday'] = pd.to_datetime(df['i_visit_start_time'], unit='us').dt.weekday
df_cp2['c_visit_start_hour'] = pd.to_datetime(df['i_visit_start_time'], unit='us').dt.hour
df_cp2

---

In [ ]:
columns_to_encode = [item for sublist in list(encodings.values()) for item in sublist]
columns_to_encode

In [ ]:
df.columns

In [ ]:
[item in df.columns for item in columns_to_encode]

In [ ]:
all(item in df.columns for item in columns_to_encode)

In [ ]:
[item in df.columns for item in columns_to_encode + ['added_col']]

In [ ]:
all(item in df.columns for item in columns_to_encode + ['added_col'])

---

In [ ]:
import re
from sklearn.impute import SimpleImputer

In [ ]:
df = pd.read_csv('../data/03_primary/df_train.csv')

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
num_cols = [item for item in df.columns if re.compile('^n_').match(item)]
cat_cols = [item for item in df.columns if re.compile('^c_').match(item)]

In [ ]:
df[num_cols] = df[num_cols].astype(float)
df[cat_cols] = np.where(pd.isnull(df[cat_cols]), df[cat_cols], df[cat_cols].astype(str))

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
imputation_strategies = {
    'mean': ['n_total_time_on_site'],
    'zero': [],
    'mostfreq': [],
    # 'unknown': ['c_traffic_campaign', 'c_weekday', 'c_visit_start_hour'],
    'unknown': ['c_traffic_campaign'],
}

In [ ]:
df_cp = df.copy()

In [ ]:
if len(imputation_strategies['mean']) > 0:
    num_mean_imputer = SimpleImputer(strategy='mean')
    num_mean_imputer.fit(df[imputation_strategies['mean']])
else:
    num_mean_imputer = None

if len(imputation_strategies['zero']) > 0:
    num_zero_imputer = SimpleImputer(strategy='constant', fill_value=0.)
    num_zero_imputer.fit(df[imputation_strategies['zero']])
else:
    num_zero_imputer = None

if len(imputation_strategies['mostfreq']) > 0:
    cat_mostfreq_imputer = SimpleImputer(strategy='most_frequent')
    cat_mostfreq_imputer.fit(df[imputation_strategies['mostfreq']])
else:
    cat_mostfreq_imputer = None

if len(imputation_strategies['unknown']) > 0:
    cat_unknown_imputer = SimpleImputer(strategy='constant', fill_value='UNKNOWN')
    cat_unknown_imputer.fit(df[imputation_strategies['unknown']])
else:
    cat_unknown_imputer = None

In [ ]:
if num_mean_imputer is not None:
    df_cp[num_mean_imputer.feature_names_in_] = num_mean_imputer.transform(df_cp[num_mean_imputer.feature_names_in_])

if num_zero_imputer is not None:
    df_cp[num_zero_imputer.feature_names_in_] = num_zero_imputer.transform(df_cp[num_zero_imputer.feature_names_in_])

if cat_mostfreq_imputer is not None:
    df_cp[cat_mostfreq_imputer.feature_names_in_] = cat_mostfreq_imputer.transform(df_cp[cat_mostfreq_imputer.feature_names_in_])

if cat_unknown_imputer is not None:
    df_cp[cat_unknown_imputer.feature_names_in_] = cat_unknown_imputer.transform(df_cp[cat_unknown_imputer.feature_names_in_])

In [ ]:
df_cp.isnull().sum()

In [ ]:
df_cp

In [ ]:
df_test = df = pd.read_csv('../data/03_primary/df_test.csv')

In [ ]:
df_cp = df_test

In [ ]:
df_cp

In [ ]:
if num_mean_imputer is not None:
    df_cp[num_mean_imputer.feature_names_in_] = num_mean_imputer.transform(df_cp[num_mean_imputer.feature_names_in_])

if num_zero_imputer is not None:
    df_cp[num_zero_imputer.feature_names_in_] = num_zero_imputer.transform(df_cp[num_zero_imputer.feature_names_in_])

if cat_mostfreq_imputer is not None:
    df_cp[cat_mostfreq_imputer.feature_names_in_] = cat_mostfreq_imputer.transform(df_cp[cat_mostfreq_imputer.feature_names_in_])

if cat_unknown_imputer is not None:
    df_cp[cat_unknown_imputer.feature_names_in_] = cat_unknown_imputer.transform(df_cp[cat_unknown_imputer.feature_names_in_])

In [ ]:
df_cp

---

In [ ]:
df = pd.read_csv('../data/04_feature/df_fe_train.csv')

In [ ]:
df

In [ ]:
features_to_exclude = ['c_is_mobile', 'c_operating_system', 'c_weekday', 'n_total_time_on_site']
# features_to_exclude = []

In [ ]:
df_cp = df.copy()

In [ ]:
for feature in features_to_exclude:
    feature_list = [item for item in df.columns if re.compile(f'^{feature}').match(item)]
    df_cp = df_cp.drop(feature_list, axis=1)

In [ ]:
df_cp